The source of this notebook: https://github.com/Azure-Samples/MachineLearningSamples-DeepLearningforPredictiveMaintenance

# Step 1: Data Ingestion & Preparation

In [ ]:
# import the libraries
print("Importing the libraries is in progress...")

import os
import csv
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler

%matplotlib inline
import matplotlib.pyplot as plt
import glob
import urllib

print("Import complete successfully!")

Source of dataset: https://www.kaggle.com/arnabbiswas1/microsoft-azure-predictive-maintenance

## Data Ingestion

In [ ]:
# Sorting input file by machineID (ascending) and datetime (descending) to determine RUL
print('Sorting data is in progress...')

df1 = pd.read_csv('./data/sourceData/PdM_telemetry_model_age_07_maint_err_fail_final.csv', delimiter=',')
df1 = df1.sort_values(['machineID', 'datetime'], ascending=[True, False])
df1.to_csv('./data/intermediateData/01_PdM_telemetry_model_age_07_maint_err_fail_final_sorted_pre.csv')

with open("data/intermediateData/01_PdM_telemetry_model_age_07_maint_err_fail_final_sorted_pre.csv", "r", newline='') as source1:
    reader1 = csv.reader(source1)
      
    with open("data/intermediateData/02_PdM_telemetry_model_age_07_maint_err_fail_final_sorted.csv", "w", newline='') as result1:
        writer1 = csv.writer(result1)
        for r1 in reader1:
            writer1.writerow((r1[1], r1[2], r1[3], r1[4], r1[5], r1[6], r1[7], r1[8], r1[9], r1[10]))

source1.close()
result1.close()

print('Writing of .../data/intermediateData/02_PdM_telemetry_model_age_07_maint_err_fail_final_sorted.csv completed successfully.')

In [ ]:
# Determine RUL
print('Determine RUL is in progress...')

with open("data/intermediateData/02_PdM_telemetry_model_age_07_maint_err_fail_final_sorted.csv", "r", newline='') as source2:
    reader2 = csv.reader(source2)
      
    with open("data/intermediateData/03_PdM_telemetry_model_age_07_maint_err_fail_final_RUL.csv", "w", newline='') as result2:
        writer2 = csv.writer(result2)
        writeData = 0
        mIDprev = 1
        mID = 1
        i = 0
        for r2 in reader2:
            if r2[0] == "datetime":
                writer2.writerow(r2+["RUL"])
            else:
                mID = int(r2[1])
                if mID != mIDprev:
                    writeData = 0
                    mIDprev = mID
                else:
                    if ('comp' in r2[9]) == True:
                        writeData = 1
                        i = 0
                        writer2.writerow(r2+[i])
                        mIDprev = mID
                        i = i+1
                    elif writeData == 1:
                        writer2.writerow(r2+[i])
                        mIDprev = mID
                        i = i+1

source2.close()
result2.close()

print('Writing of .../data/intermediateData/03_PdM_telemetry_model_age_07_maint_err_fail_final_RUL.csv completed successfully.')

In [ ]:
# Sorting intermediate file by machineID (ascending) and datetime (ascending)
print('Sorting data is in progress...')

df3 = pd.read_csv('./data/intermediateData/03_PdM_telemetry_model_age_07_maint_err_fail_final_RUL.csv', delimiter=',')
df3 = df3.sort_values(['machineID', 'datetime'], ascending=[True, True])
df3.to_csv('./data/intermediateData/04_PdM_telemetry_model_age_07_maint_err_fail_final_RUL_asc_pre.csv')

with open("data/intermediateData/04_PdM_telemetry_model_age_07_maint_err_fail_final_RUL_asc_pre.csv", "r", newline='') as source3:
    reader3 = csv.reader(source3)
      
    with open("data/intermediateData/05_PdM_telemetry_model_age_07_maint_err_fail_final_asc.csv", "w", newline='') as result3:
        writer3 = csv.writer(result3)
        for r3 in reader3:
            writer3.writerow((r3[1], r3[2], r3[3], r3[4], r3[5], r3[6], r3[7], r3[8], r3[9], r3[10], r3[11]))

source3.close()
result3.close()

print('Writing of .../data/intermediateData/05_PdM_telemetry_model_age_07_maint_err_fail_final_asc.csv completed successfully.')

In [ ]:
# Determine cycle
print('Determine cycle is in progress...')

with open("data/intermediateData/05_PdM_telemetry_model_age_07_maint_err_fail_final_asc.csv", "r", newline='') as source4:
    reader4 = csv.reader(source4)
      
    with open("data/intermediateData/06_PdM_telemetry_model_age_07_maint_err_fail_final_cycle.csv", "w", newline='') as result4:
        writer4 = csv.writer(result4)
        mIDprev = 1
        mID = 1
        i = 1
        for r4 in reader4:
            if r4[0] == "datetime":
                writer4.writerow((r4[0], r4[1], str('cycle'), r4[2], r4[3], r4[4], r4[5], r4[6], r4[7], r4[8], r4[9], r4[10]))
            else:
                mID = int(r4[1])
                if mID != mIDprev:
                    i = 1
                    writer4.writerow((r4[0], r4[1], str(i), r4[2][-1:], r4[3], r4[4], r4[5], r4[6], r4[7], r4[8], r4[9], r4[10]))
                    mIDprev = mID
                    i = i+1
                else:
                    if r4[10] == str(0):
                        writer4.writerow((r4[0], r4[1], str(i), r4[2][-1:], r4[3], r4[4], r4[5], r4[6], r4[7], r4[8], r4[9], r4[10]))
                        i = 1
                    else:
                        writer4.writerow((r4[0], r4[1], str(i), r4[2][-1:], r4[3], r4[4], r4[5], r4[6], r4[7], r4[8], r4[9], r4[10]))
                        i = i+1

source4.close()
result4.close()

print('Writing of .../data/intermediateData/06_PdM_telemetry_model_age_07_maint_err_fail_final_cycle.csv completed successfully.')

In [ ]:
# split data into train and test data
print('Split data into train and test data is in progress...')

# write train data
with open("data/intermediateData/06_PdM_telemetry_model_age_07_maint_err_fail_final_cycle.csv", "r", newline='') as source5:
    reader5 = csv.reader(source5)
      
    with open("data/preparedData/PdM_telemetry_model_age_07_maint_err_fail_final_RUL_train.csv", "w", newline='') as result5:
        writer5 = csv.writer(result5)
        for r5 in reader5:
            if r5[1] not in (str(92),str(99),str(97),str(98),str(89),str(93),str(91),str(100)):
                writer5.writerow(r5)
            
source5.close()
result5.close()

print('Writing of .../data/preparedData/PdM_telemetry_model_age_07_maint_err_fail_final_RUL_train.csv completed successfully.')

# write test data
with open("data/intermediateData/06_PdM_telemetry_model_age_07_maint_err_fail_final_cycle.csv", "r", newline='') as source6:
    reader6 = csv.reader(source6)
      
    with open("data/intermediateData/PdM_telemetry_model_age_07_maint_err_fail_final_RUL_test.csv", "w", newline='') as result6:
        writer6 = csv.writer(result6)
        for r6 in reader6:
            if r6[0] == "datetime" or r6[1] in (str(92),str(99),str(97),str(98),str(89),str(93),str(91),str(100)):
                writer6.writerow(r6)
            
source6.close()
result6.close()

print('Writing of .../data/intermediateData/PdM_telemetry_model_age_07_maint_err_fail_final_RUL_test.csv completed successfully.')

print('Split data into train and test data completed successfully.')

In [ ]:
# read training data 
train_df = pd.read_csv('./data/intermediateData/PdM_telemetry_model_age_07_maint_err_fail_final_RUL_train.csv', sep=",", header=0)
train_df.head()

In [ ]:
# read test data
test_df = pd.read_csv('./data/intermediateData/PdM_telemetry_model_age_07_maint_err_fail_final_RUL_test.csv', sep=",", header=0)
test_df.head()

## Data Preprocessing

Using RUL, we can create a label indicating time to failure. We define a boolean (`True\False`) value for `label1` indicating the engine will fail within 30 days (RUL $<= 30$). We can also define a multiclass `label2` $\in \{0, 1, 2\}$ indicating {Healthy, RUL <=30, RUL <=15} cycles. 

In [ ]:
# generate label columns for training data
w1 = 30
w0 = 15

# Label1 indicates a failure will occur within the next 30 cycles.
# 1 indicates failure, 0 indicates healthy 
train_df['label1'] = np.where(train_df['RUL'] <= w1, 1, 0 )

# label2 is multiclass, value 1 is identical to label1,
# value 2 indicates failure within 15 cycles
train_df['label2'] = train_df['label1']
train_df.loc[train_df['RUL'] <= w0, 'label2'] = 2
train_df.head()

In the [Predictive Maintenance Template](https://gallery.cortanaintelligence.com/Collection/Predictive-Maintenance-Template-3) , cycle column is also used for training so we will also include the cycle column. Here, we normalize the columns in the training data.

In [ ]:
# MinMax normalizatio - train - step 1
train_df['cycle_norm'] = train_df['cycle']
train_df.drop(train_df.columns[[0]], axis=1, inplace=True)
train_df.drop(train_df.columns[[8]], axis=1, inplace=True)
train_df.drop(train_df.columns[[8]], axis=1, inplace=True)
train_df.head()

In [ ]:
# MinMax normalization - train - step 2
cols_normalize = train_df.columns.difference(['machineID','cycle','RUL','label1','label2'])
min_max_scaler = MinMaxScaler()
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
                             columns=cols_normalize, 
                             index=train_df.index)
join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
train_df = join_df.reindex(columns = train_df.columns)
train_df.head()

Next, we prepare the test data. We normalize the data using the same parameters from the training data normalization.

In [ ]:
# MinMax normalizatio - test - step 1
test_df['cycle_norm'] = test_df['cycle']
test_df.drop(test_df.columns[[0]], axis=1, inplace=True)
test_df.drop(test_df.columns[[8]], axis=1, inplace=True)
test_df.drop(test_df.columns[[8]], axis=1, inplace=True)
test_df.head()

In [ ]:
# MinMax normalizatio - test - step 2
cols_normalize = test_df.columns.difference(['machineID','cycle','RUL','label1','label2'])
min_max_scaler = MinMaxScaler()
norm_test_df = pd.DataFrame(min_max_scaler.fit_transform(test_df[cols_normalize]), 
                             columns=cols_normalize, 
                             index=test_df.index)
join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
test_df = join_df.reindex(columns = test_df.columns)
test_df.head()

We then create the same labels as used for the `training` data.

In [ ]:
# generate label columns w0 and w1 for test data
test_df['label1'] = np.where(test_df['RUL'] <= w1, 1, 0 )
test_df['label2'] = test_df['label1']
test_df.loc[test_df['RUL'] <= w0, 'label2'] = 2
test_df.head()

## Data Visualization
One critical advantage of LSTMs is their ability to remember from long-term sequences (window sizes) which is hard to achieve by traditional feature engineering as computing rolling averages over large window sizes (i.e. 50 cycles) may lead to loss of information due to smoothing and abstracting of values over such a long period. While feature engineering over large window sizes may not make sense, LSTMs are able to use all the information in the window as input.

We first look at an example of the sensor values for 50 cycles prior to the failure for engine `id = 3`. 

In [ ]:
# preparing data for visualizations 
# window of 50 cycles prior to a failure point for machibe id 89
engine_id89 = test_df[test_df['machineID'] == 89]
engine_id89_50cycleWindow = engine_id89[engine_id89['RUL'] <= engine_id89['RUL'].min() + 50]
cols1 = ['volt', 'rotate']
engine_id89_50cycleWindow1 = engine_id89_50cycleWindow[cols1]
cols2 = ['pressure', 'vibration']
engine_id89_50cycleWindow2 = engine_id89_50cycleWindow[cols2]

# plotting sensor data for machine ID 89 prior to a failure point 
ax1 = engine_id89_50cycleWindow1.plot(subplots=True, sharex=True, figsize=(20,20))

In [ ]:
# plotting sensor data for machine ID 89 prior to a failure point 
ax2 = engine_id89_50cycleWindow2.plot(subplots=True, sharex=True, figsize=(20,20))

## Persist the data sets

The `Code\2_model_building_and_evaluation.ipynb` Jupyter notebook will read these data files and train a LSTM network to predict the probability of machine failure within the next 30 cycles using the previous n cycles.

In [ ]:
print('Data saving into files in progress...')

train_df.to_csv('./data/preparedData/PM_train_prepared.csv', encoding='utf-8', index=False)
test_df.to_csv('./data/preparedData/PM_test_prepared.csv', encoding='utf-8', index=False)

print("Data files saved!")